### Imports and utility

In [25]:
# Standard library imports
import os
import sys
import importlib
import numpy as np
import random
import cv2
from tqdm import tqdm

# Add project-specific paths
sys.path.append(os.path.join(os.getcwd(), 'common'))

# Import custom modules
import utility
import embedding
import detection
import grt

# Reload module for updates
importlib.reload(utility)
importlib.reload(embedding)
importlib.reload(detection)
importlib.reload(grt)

<module 'grt' from 'c:\\Users\\Utente\\Documents\\Università\\Master\\MultimediaDataSecurity\\CTM\\multimedia-data-security\\common\\grt.py'>

In [26]:
def points(avg):
    if avg >= 53:
        points = 0
    elif avg >= 50:
        points = 1
    elif avg >= 47:
        points = 2
    elif avg >= 44:
        points = 3
    elif avg >= 41:
        points = 4
    elif avg >= 38:
        points = 5
    else:
        points = 6
    return points

### Embedding

In [27]:
# Randomly select 10 images
img_folder = 'sample_imgs'
img_files =  [f for f in os.listdir(img_folder) if f.endswith(('.bmp'))]
img_files = random.sample(img_files, 10) #---------------------------
images = []
for file in img_files:
    img_path = os.path.join(img_folder, file)
    images.append(cv2.imread(img_path, 0))
images = np.array(images) # optional

mark = np.load('ammhackati.npy')

alpha = 2.98
max_layer = 1

watermarked = []
for img in images:
    watermarked.append(embedding.embedding(img, mark, alpha, max_layer))
    
avg = 0
max = 0

for index, img in enumerate(images):
    wpsnr = utility.wpsnr(img, watermarked[index])
    avg += wpsnr
    if max < wpsnr:
        max = wpsnr
        
avg = avg / len(watermarked)
print("Average wpsnr of successful attacks: " + str(avg))
print("Max wpsnr of successful attacks: ", max)    


Average wpsnr of successful attacks: 48.98690298483674
Max wpsnr of successful attacks:  54.8349102118612


### Global attacks

1. Blur using gaussian filters

In [28]:
progress_bar = tqdm(watermarked, desc = "Blurring 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.1, dtype = np.float16):
        attacked, attack_name, usd = grt.blur_gauss(wm, sigma)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 

Blurring 10 images: 100%|██████████| 10/10 [00:38<00:00,  3.87s/it]

Average wpsnr of successful attacks: 141.07994643502278 --> 0 robustness points
Max wpsnr of successful attacks:  147.52561902277213
{'image': '0028.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 142.61356730889204, 'detection': 0}
{'image': '0100.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 138.0045341420464, 'detection': 0}
{'image': '0017.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 142.1489024253669, 'detection': 0}
{'image': '0020.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 140.49474953128498, 'detection': 0}
{'image': '0042.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 141.28577991887548, 'detection': 0}
{'image': '0033.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 134.80465789108766, 'detection': 0}
{'image': '0093.bmp', 'attack': 'blur_gauss', 'param': {'sigma': 0.2}, 'wpsnr': 143.45673394600897, 'detection': 0}
{'image': '0051.bmp', 'attack': 'blur_gauss', 'param': {'

2. Blur using median filters

In [19]:
progress_bar = tqdm(watermarked, desc = "Blurring 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for height in np.arange(1, 15, 2):
        for width in np.arange(1, 15, 2):
            attacked, attack_name, usd = grt.blur_median(wm, [height, width])
            detected = detection.detection(images[index], wm, attacked, alpha, 2)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

Blurring 10 images: 100%|██████████| 10/10 [02:28<00:00, 14.80s/it]

Average wpsnr of successful attacks: 35.1098714306007 --> 6 robustness points
Max wpsnr of successful attacks:  51.531375520108725
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [1, 3]}, 'wpsnr': 42.94181379127302, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [3, 1]}, 'wpsnr': 42.41057277250907, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [5, 1]}, 'wpsnr': 36.63558127075727, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [7, 1]}, 'wpsnr': 34.02435570303921, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [9, 1]}, 'wpsnr': 32.20564374262708, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [11, 1]}, 'wpsnr': 31.047188575188795, 'detection': 0}
{'image': '0023.bmp', 'attack': 'blur_median', 'param': {'size': [13, 1]}, 'wpsnr': 30.227983732955067, 'detection': 0}
{'image': '0089.bmp', 'attack': 'blur_m

3. Adding white gaussian noise (AWGN)

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying awgn on 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for std in np.arange(0, 1, 0.02, dtype = np.float16):
        attacked, attack_name, usd = grt.awgn(wm, 0, std, 100)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

4. JPEG compression

In [12]:
progress_bar = tqdm(watermarked, desc = "Compressing 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for qf in range(1, 100, 1):
        attacked, attack_name, usd = grt.jpeg_compression(wm, (100 - qf))
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

Compressing 10 images:   0%|          | 0/10 [00:00<?, ?it/s]

Compressing 10 images: 100%|██████████| 10/10 [00:18<00:00,  1.81s/it]

Average wpsnr of successful attacks: 63.027913299829905 --> 0 robustness points
Max wpsnr of successful attacks:  68.67201270085079
{'image': '0037.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 62.763859877640975, 'detection': 0}
{'image': '0065.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 63.375881692235325, 'detection': 0}
{'image': '0097.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 57.9848183353068, 'detection': 0}
{'image': '0074.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 68.67201270085079, 'detection': 0}
{'image': '0015.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 58.446276307419296, 'detection': 0}
{'image': '0071.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 64.73134952989393, 'detection': 0}
{'image': '0086.bmp', 'attack': 'jpeg_compression', 'param': {'qf': 99}, 'wpsnr': 66.56274158242138, 'detection': 0}
{'image': '0091.bmp', 'attack': 'jpeg_compressi

5. Resizing

In [ ]:
progress_bar = tqdm(watermarked, desc = "Resizing 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for scale in np.arange(0, 1, 0.001, dtype = np.float16):
        attacked, attack_name, usd = grt.resize(wm, (1 - scale))
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 

### Localized attacks

1. Blur using gaussian filters only on edges

In [8]:
progress_bar = tqdm(watermarked, desc = "Blurring 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.1, dtype = np.float16):
        for edge_func in range(0, 1): #----------
            attacked, attack_name, usd = grt.gauss_edge(wm, sigma, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, 2)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 

Blurring 10 images: 100%|██████████| 10/10 [15:44<00:00, 94.46s/it]

Average wpsnr of successful attacks: 40.63276143478812 --> 5 robustness points
Max wpsnr of successful attacks:  151.91427421969775
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.2, 'edge_func': 0}, 'wpsnr': 140.82029161797908, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.2998, 'edge_func': 0}, 'wpsnr': 80.55965736911098, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.4, 'edge_func': 0}, 'wpsnr': 60.13088489914072, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.5, 'edge_func': 0}, 'wpsnr': 51.74579519760719, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.5996, 'edge_func': 0}, 'wpsnr': 47.71758027264661, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.6997, 'edge_func': 0}, 'wpsnr': 45.22940334912825, 'detection': 0}
{'image': '0030.bmp', 'attack': 'gauss_edge', 'param': {'sigma': 0.8, 'e